In [ ]:
# Import python packages
import streamlit as st
import pandas as pd
import os
import logging
import datetime
from snowflake.snowpark.functions import when_matched, when_not_matched
# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [ ]:
def fix_date_cols(df, tz = 'UTC'):
    cols = df.select_dtypes(include=['datetime64[ns]']).columns
    for col in cols:
        df[col] = df[col].dt.tz_localize(tz)

In [ ]:
logger = logging.getLogger('default')
# if environment_level == 'NON-PROD':
#     logger.setLevel(logging.DEBUG)    

In [ ]:
stage_path = '@ingestion_staging'
database = 'ADVENTUREWORKS_DW'
schema = 'BRONZE'

In [ ]:
stage_table = session.sql(f""" LS {stage_path}""").collect()
stage_table

In [ ]:
stage_df =pd.DataFrame(stage_table)
tables = []
for item in stage_df['name']:
    table_name = item[17:]
    table_name_parts = table_name.split('.')
    tables.append((table_name_parts[1], table_name))
tables


In [ ]:
stage_path = '@ingestion_staging'
table_data={}
for item in tables:
    table_data[item[0]] = session.read.options({"FIELD_OPTIONALLY_ENCLOSED_BY": '"', "PARSE_HEADER": True, "INFER_SCHEMA" : True}).csv(stage_path + item[1])
    # table_data[item[0]].show()

In [ ]:
SELECT TABLE_SCHEMA, TABLE_NAME FROM ADVENTUREWORKS_DW.INFORMATION_SCHEMA.TABLES WHERE TABLES.TABLE_SCHEMA = 'BRONZE'

In [ ]:
tables_exist_df = Get_Tables.to_pandas()
tables_exist_df.head()

In [ ]:
print(table_data.keys())

In [ ]:
# if 'Territory' in tables_exist_df['TABLE_NAME'].values:
#     print('true')
# else:
#     print('false')
# print(table_data['Territory']['TerritoryID'])
# target = session.table(database+ '.' + schema + '.' + '"ProductCategory"')
# print(target['ProductCategoryID'])
print(table_data.keys())
# print(table_data['Territory'].to_pandas().describe)
# print(table_data['Territory']['"TerritoryID"'])
# table_data['Product'].index
print(target.columns)
test_df = table_data['Product'].to_pandas().head()
type(test_df['SellStartDate'][0])

In [ ]:
for table in table_data.keys():
    if table not in tables_exist_df['TABLE_NAME'].values:
        write_df = table_data[table].to_pandas()
        fix_date_cols(write_df)
        obj = session.write_pandas( write_df,
                                    table_name = table, 
                                    database = database,
                                    schema = schema,
                                    auto_create_table = True,
                                    quote_identifiers = True,
                                    use_logical_type = True)
    else:
        update_dict= {}
        target = session.table(database+ '.' + schema + '."' + table+'"')
        cols = table_data[table].columns[1:]
        # print(table_data[table].columns)
        print(cols)
        #print(target.columns)
        # print(target.columns)
        # print(target['"'+ table + 'ID' + '"'])
        print(table_data[table]['"'+ table + 'ID' + '"'])

        
        for col in cols:
            update_dict[col] = table_data[table][col]
        print(update_dict)
        target.merge(table_data[table], 
                     (target['"'+ table + 'ID' + '"'] == table_data[table]['"'+ table + 'ID' + '"']),
              [when_matched().update(update_dict), #fixme build dict of all but id column matching to source
              when_not_matched().insert({'"'+ table + 'ID' + '"':table_data[table]['"'+ table + 'ID' + '"']})
              ])
        #needs to have snowparkdf as source for merge, pandas as source for write, need way to ingest csv as snowpark tables